# **Tutorial** - (semi)-supervised topic modeling
(last updated 11-09-2022)

In this tutorial, we will be looking at a new feature of BERTopic, namely (semi)-supervised topic modeling! This allows us to steer the dimensionality reduction of the embeddings into a space that closely follows any labels you might already have. 

## Semi-supervised modeling
(semi)-supervised topic modeling is a class of methods that allows the user to perform topic modeling with previously defined labels. This might help nudge the model towards specific topics or classes for which you have labels. 

<br>

<img src="https://raw.githubusercontent.com/MaartenGr/BERTopic/master/images/logo.png" width="40%">

# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# Installing BERTopic

We start by installing BERTopic from PyPi:

In [ ]:
%%capture
!pip install bertopic

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

# **Data**
For this example, we use the popular 20 Newsgroups dataset which contains roughly 18000 newsgroups posts that each is assigned to one of 20 topics:

In [ ]:
import pandas as pd
from bertopic import BERTopic
from sklearn.datasets import fetch_20newsgroups

data = fetch_20newsgroups(subset='all',  remove=('headers', 'footers', 'quotes'))
docs = data["data"]
targets = data["target"]
target_names = data["target_names"]
classes = [data["target_names"][i] for i in data["target"]]

Each document can be put into one of the following categories:

In [ ]:
target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

# **(semi)-Supervised modeling**


## Basic Model
Before we start with semi-supervised modeling, let us first take a look at the output of the basic model.

In [ ]:
topic_model = BERTopic(verbose=True)
topics, _ = topic_model.fit_transform(docs)

2021-04-25 05:59:40,718 - BERTopic - Transformed documents to Embeddings


2021-04-25 05:59:56,400 - BERTopic - Reduced dimensionality with UMAP
2021-04-25 05:59:57,376 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [ ]:
topic_model.get_topic_info().head(10)

,Topic,Count,Name
0,-1,6714,-1_to_is_the_of
1,0,1829,0_game_team_games_he
2,1,576,1_key_clipper_chip_encryption
3,2,526,2_ites_cheek_yep_huh
4,3,469,3_israel_israeli_jews_arab
5,4,444,4_drive_scsi_drives_ide
6,5,423,5_fbi_batf_koresh_fire
7,6,360,6_car_engine_cars_ford
8,7,188,7_ram_sale_drive_meg
9,8,182,8_bike_tires_tire_bikes


The topics that were created mostly make sense. There are some clearly defined topics such as "nasa, orbit, spacecraft, moon" but also some topics that seem mostly derived from other topics. We can visualize this by extracting the topic representations per class and see if our unsupervised model closely resembles this. 

**NOTE**: You can **hover** over the bars to see the representation per class!!

In [ ]:
topics_per_class = topic_model.topics_per_class(docs, classes=classes)
fig_unsupervised = topic_model.visualize_topics_per_class(topics_per_class, top_n_topics=10)
fig_unsupervised

20it [00:06,  2.93it/s]


The results do seem promising. Topics like "nasa, space, etc" seem to be clearly related to sci.space, but some topics were created that span many categories. For example, we expect the topic "bike, bikes, etc"  to only appear in rec.motorcycles.  

## Semi-supervised
In the example above you might notice that some topics were somewhat smushed together. What we would like to see is a clear separation between those topics. Fortunately, we have to labels and can use them to improve the model. 

Since we are not interested in any other topics, this method is called semi-supervised topic modeling. In practice, this means that we have the labels of some documents but not all. 

For this example let's say we only have the labels of all computer-related categories:

In [ ]:
 labels_to_add = ['comp.graphics', 'comp.os.ms-windows.misc',
                  'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware',
                  'comp.windows.x',]
 indices = [target_names.index(label) for label in labels_to_add]
 new_labels = [label if label in indices else -1 for label in targets]

When generating our new labels it is important to mark unknown classes as **-1**. Next, we use those newly constructed labels to again run BERTopic:

In [ ]:
topic_model = BERTopic(verbose=True)
topics, _ = topic_model.fit_transform(docs, y=new_labels)

2021-04-25 05:55:43,262 - BERTopic - Transformed documents to Embeddings


2021-04-25 05:56:03,260 - BERTopic - Reduced dimensionality with UMAP
2021-04-25 05:56:05,695 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [ ]:
topic_model.get_topic_info().head(10)

,Topic,Count,Name
0,-1,5445,-1_the_of_to_and
1,0,1815,0_game_team_he_games
2,1,875,1_window_server_motif_widget
3,2,861,2_image_jpeg_images_graphics
4,3,593,3_key_clipper_chip_encryption
5,4,550,4_windows_dos_file_files
6,5,529,5_ites_cheek_kidding_yep
7,6,489,6_israel_israeli_jews_arab
8,7,445,7_fbi_batf_fire_koresh
9,8,400,8_patients_medical_disease_doctor


Finally, we can again extract the topics per class to see if our semi-supervised approach had some effect:

In [ ]:
topics_per_class = topic_model.topics_per_class(docs, classes=classes)
fig_semi_supervised = topic_model.visualize_topics_per_class(topics_per_class, top_n_topics=10, width=900)
fig_semi_supervised

We can clearly see that many more topics about computers were created and that the seperation between those topics are solid. This indicates that even if you do not have all the labels, you can definitely improve the model!

However, there are still some clusters that could be improved with the labels that we have. 

## Supervised

Finally, we are going to be using all labels. These labels help BERTopic understand where most clusters can be found. However, this does not mean that it will only find the 20 clusters that we have defined. If there are sub-clusters to be found, then there is a good chance BERTopic will find them! 

In [ ]:
topic_model = BERTopic(verbose=True)
topics, _ = topic_model.fit_transform(docs, y=targets)

Batches:   0%|          | 0/589 [00:00<?, ?it/s]

2022-09-11 08:16:00,180 - BERTopic - Transformed documents to Embeddings
2022-09-11 08:16:22,467 - BERTopic - Reduced dimensionality
2022-09-11 08:16:23,475 - BERTopic - Clustered reduced embeddings


In [ ]:
topic_model.get_topic_info().head(10)

,Topic,Count,Name
0,-1,3426,-1_to_of_the_and
1,0,948,0_space_launch_nasa_orbit
2,1,912,1_car_cars_engine_it
3,2,906,2_he_game_year_baseball
4,3,906,3_window_server_motif_widget
5,4,866,4_mac_apple_drive_monitor
6,5,844,5_gun_guns_firearms_the
7,6,788,6_team_game_25_hockey
8,7,706,7_image_jpeg_images_graphics
9,8,705,8_key_clipper_encryption_chip


Not only do we see a nice seperation of the topics, there are significantly less outliers which shows that BERTopic has improved in connecting the documents to topics. 

Let's see the results by again visualizing the topic representation per class:

In [ ]:
topics_per_class = topic_model.topics_per_class(docs, classes=classes)
fig_supervised = topic_model.visualize_topics_per_class(topics_per_class, top_n_topics=10, width=900)
fig_supervised

Now that we have used all labels, BERTopic seems to closely match our pre-defined labels. Moreover, it still allows to discover topics that were not previously defined. Thus, you can use this method to find unknown topics in pre-defined topics!